In [1]:
# %load query.py
import numpy as np
from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
import requests

In [2]:
# %load query.py
import numpy as np
import glob

from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
from astropy.table import Table
from astropy.coordinates import SkyCoord, ICRS, Galactic, Angle

In [3]:
files = glob.glob('../IPTA_DR1/*/*.par')

In [4]:
RAs = []
DECs = []
Names = []
pmra = []
pmdec = []
pepoch = []
posepoch = []
dmepoch = []
dm = []

for file in files:
    d = {}
    with open(file) as f:
        for line in f:
            key = line.split()[0]
            value = line.split()[1]
            d[key] = value
            
    try:
        RAs.append(Angle(d['RAJ'], u.hourangle).degree)
        #print(d['PSRJ'], d['DECJ'])
        DECs.append(Angle(d['DECJ'], u.degree).value)
        Names.append(d['PSRJ'])
        pmra.append(d['PMRA'])
        pmdec.append(d['PMDEC'])
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])
        
    except KeyError:
        coord = Galactic(l=float(d['ELONG'])*u.deg, b=float(d['ELAT'])*u.deg, pm_l_cosb = float(d['PMELONG'])*np.cos(float(d['ELAT'])*np.pi/180.)*u.mas/u.yr, pm_b=float(d['PMELAT'])*u.mas/u.yr)
        icrscoord = coord.transform_to(ICRS)
        #print(icrscoord)
        #print(d['PSRJ'])
        RAs.append(icrscoord.ra.to(u.deg).value)
        DECs.append(icrscoord.dec.to(u.deg).value)
        Names.append(d['PSRJ'])
        pmra.append(icrscoord.pm_ra_cosdec/np.cos(icrscoord.dec.to(u.rad)))
        pmdec.append(icrscoord.pm_dec)
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])        
        #coord = SkyCoord(d[''])
        
        continue

t = Table([Names, RAs, DECs, pmra, pmdec, pepoch, posepoch, dmepoch, dm], names=['names', 'ra', 'dec', 'pmra', 'pmdec', 'dm', 'pepoch', 'posepoch', 'dmepoch'])        
tmin = Table([RAs, DECs], names=['ra', 'dec'])


In [9]:
raAng = [Angle(ra, u.hourangle).degree for ra in RAs]
print(raAng)

[114.21427636874998, 128.86463933124998, 517.8973298749999, 1039.7435109335686, 1388.3497163312497, 1401.4984796999997, 1430.1321485999997, 1619.6368203312497, 1766.8222081124993, 2027.7470680687495, 2297.089845618749, 982.8453201782852, 2342.4172112124998, 2421.8866913249994, 3359.2481172062494, 3603.2439585249995, 3613.4798437499994, 3751.046551924999, 3763.6350928519673, 3876.845823512498, 3904.0936194250003, 3938.8542586187486, 3947.98542011875, 3970.872898306249, 3991.8379713624995, 4018.293324681249, 4056.942083906249, 4057.8334524499996, 4066.3208184874993, 4142.000494593749, 4213.828869424999, 4252.332423149999, 4266.024412593749, 4311.714598456249, 4313.106338924999, 4319.700292287499, 4319.330145331249, 4345.5020701375, 4423.660076293749, 4482.9922338125, 4540.37003979375, 4573.246305987498, 4625.469635562498, 4817.740488556248, 4835.1729355, 4896.903787437499, 5061.930338581249, 5239.327288387499, 5258.895952168749]


In [10]:
t.write('ipta_data.csv', format='csv', overwrite=True)
tmin.write('ipta_data_nonames.csv', format='csv', overwrite=True)

In [11]:
r = requests.post(
'http://cdsxmatch.u-strasbg.fr/xmatch/api/v1/sync',
data={'request': 'xmatch', 'distMaxArcsec': 1.0, 'RESPONSEFORMAT': 'csv',
'cat2': 'vizier:I/337/gaia', 'colRA1': 'ra', 'colDec1': 'dec'},
files={'cat1': open('ipta_data.csv', 'r')})

In [12]:
print(r.text)

angDist,names,ra,dec,pmra,pmdec,dm,pepoch,posepoch,dmepoch,ra_ep2000,dec_ep2000,errHalfMaj,errHalfMin,errPosAng,ra,dec,source_id,ref_epoch,ra_error,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_variable_flag,l,b
0.970148,J0437-4715,69.31623406223791,-47.25253075275593,121.43993281718667,-71.47620158262019,54500,54500,54500,2.644571992220118,69.3165765588,-47.2526670522,0.003,0.002,0,69.3165765588,-47.2526670522,4789864076732331648,2015.0,1.992,2.741,,,,,,,-0.671,0,65,1.28203e+02,2.46615e+00,20.255,NOT_AVAILABLE,253.3946170111,-41.9631552362
0.140141,J1012+5307,153.13932304124995,53.1173056,2.6237122562025093,-25.458808920674365,55000,55000,55000,9.021803950562022,153.1393295862,53.1172668707,0.001,0.000,0,153.1393295862,53.1172668707,851610857095609984,2015.0,0.395,0.614,,,,,,,0.058,0,148,2.43759e+02,1.93173e+00,19.557,NOT_AVAILABLE,160.3471938161,50.8578532615


In [13]:
h = open('ipta_crossmatch_1arcsecond.csv', 'w')
h.write(r.text)
h.close()